In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
from io import BytesIO
from google.cloud import storage

In [3]:
bs = 64

In [15]:
#Random tranform for training model
tfms = get_transforms()
d_path = Path('data')

pd.read_csv(d_path/'labels.csv').head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [16]:
data = ImageDataBunch.from_csv(d_path, ds_tfms=tfms)
data.show_batch(rows=3, figsize=(5,5))

FileNotFoundError: [Errno 2] No such file or directory: 'data/Train_1'